# Algorithmes Évolutionnaires — Travaux dirigés N◦6 : neuroévolution

Dans cette séance, nous allons développer un algorithme évolutionnaire du type NEAT pour optimiser un réseau de neurones. Pour ce faire, nous utiliserons les frameworks DEAP et Keras.

### 1 Introduction
Pour appliquer **les algorithmes évolutionnaires aux réseaux de neurones**, il y a deux possibilités : soit on code soi-même les primitives du réseau de neurones, soit on s’appuye sur un frameworks, comme **Keras**, qui fournit toutes ces primitives et plein de choses en plus. 
Dans les deux cas, il y a des avantages et des inconvénients : 
* si on décide de coder soi-même, il faudra investir du temps dans le développement, mais on aura un contrôle et une compréhension parfaite sur le code produit ; 
* si on opte pour un framework, il faudra investir du temps pour apprendre à l’utiliser, mais on aura accès à tout ce qu’il y a de plus récent et performant en matière d’apprentissage profond

* Le problème que nous souhaitons rédoudre grâce aux **algorithmes évolutionnaires** est le suivant : 
    - Ce jeu de données décrit les dossiers médicaux des Indiens Pima et indique si chaque patient aura ou non un début de diabète dans les cinq ans.
    - Nous souhaitons donc trouver un modèle capable de prédire si oui ou non un patient sera donc atteind d'un début de diabète dans les cinq ans.

* Lien KAGGLE des data :  https://www.kaggle.com/kumargh/pimaindiansdiabetescsv

* Les champs sont décrits ci-dessous :

**Variables d'entrée (X)** 

    preg = Nombre de fois enceinte
    plas = Concentration de glucose plasmatique à 2 heures dans un test de tolérance au glucose par voie orale
    pres = Pression artérielle diastolique (mm Hg)
    skin = Épaisseur du pli cutané du triceps (mm)
    test = Insuline sérique de 2 heures (mu U/ml)
    mass = Indice de masse corporelle (poids en kg/(taille en m)^2)
    pedi = Fonction pedigree du diabète
    age = Âge (années)

**Variables de sortie (y)**

    class = Variable de classe (1:test positif pour le diabète, 0 : test négatif pour le diabète)


In [1]:
# load the dataset as dataframe
import pandas as pd
df= pd.read_csv('pima-indians-diabete.csv', header=None)
df.head(5)

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
# load the dataset as array
from numpy import loadtxt
dataset = loadtxt('pima-indians-diabete.csv', delimiter=',')
dataset

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

### NeuroEvolution of Augmenting Topologies (NEAT)
- Démarrer avec des topologies aléatoires minimales.
- Augmenter les topologies au fur et à mesure si nécessaire.
- Suivez les gènes correspondants pour atténuer le problème des conventions concurrentes.
- Protéger les innovations par la spéciation.

<font color='green'> **La fonction de fitness** sera basée sur la **fonction de perte** du réseau de neurones.
Une fonction de perte, ou **Loss function**, est une fonction qui **évalue l’écart entre les prédictions réalisées par le réseau de neurones et les valeurs réelles des observations utilisées pendant l’apprentissage**. Plus le résultat de cette fonction **est minimisé**, plus le réseau de neurones **est performant**. Sa minimisation, c’est-à-dire réduire au minimum l’écart entre la valeur prédite et la valeur réelle pour une observation donnée, se fait en ajustant les différents poids du réseau de neurones.</font>

In [3]:
from keras.models import Sequential
from keras.layers import Dense

In [4]:
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

In [5]:
# import the necessary packages
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation
#from keras.optimizers import SGD
from keras.layers import Dense
from keras.utils import np_utils
from imutils import paths
import numpy as np
import argparse
import cv2
import os

#### Not fully connected layers model

* On cré un modèle **"Not fully connected"**, ce qui signifie que les neurones d'une couche ne communiquent par tous avec les neurones de la couche suivante.
* Nous devons créer un modèle de ce type car les méthodes de **"NEAT Structural Mutation"** (c'est à dire les mutation), se font en modifiant ces connexions. 

RAPPEL DES CONSTANTES DE DEPART
------------------------------------------------------------------------

* **MAXGEN** = Nombre maximum de générations = nombre max de population de réseaux de neurones testées
* **POPSIZE** = Taille de la population = Nombre de réseau de neurones dans la population (d'individus)
* **PMUT** = Probabilité d'une mutation
* **PX** = Probabilité d'une recombination
* **N** = Taille d'un génôme = nombre de couches du réseaux

RAPPEL DES OPERATEURS 
-----------------------------------------------------------
* Mutation : NEAT Structural Mutation 
    - Ajout de neurones
    - Ajout de connexions
* Recombinaison : NEAT Crossover 
    - Création d'un nouveau réseau de neurones à partir de deux réseaux parents
* Fitness : Loss Function 
    - Minimiser la perte 


In [6]:
# On définie le modele 

# couche d'input : 
input = keras.layers.Input(shape=(8,))

# couches cachées
hidden1 = Dense(8, activation='relu')(input)
hidden2 = Dense(8, activation='sigmoid')(hidden1)

hidden3_1 =  Dense(2)(hidden2[:,0:4])  # prend les 4 premiers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones
hidden3_2 =  Dense(2)(hidden2[:,4:])  # prend les 4 derniers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones

hidden_3 = keras.layers.concatenate([hidden3_1, hidden3_2])  
#la troisieme couche cachée possède maintenant ses 2 premiers neurones connectés au 4 premiers de la couche précédente
#et ces 2 derniers neurones connectés aux 4 derniers de la couche précédente

# output
output = Dense(1)(hidden_3) 

#on concatene les couches d'output pour en faire une seule couche output
model = keras.models.Model(input, output)

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

# evaluate the keras model
loss, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))
print('Loss : %.2f' % loss)

Epoch 1/150
77/77 [==============================] - 3s 3ms/step - loss: 1.7544 - accuracy: 0.6211
Epoch 2/150
77/77 [==============================] - 0s 3ms/step - loss: 1.4958 - accuracy: 0.6719
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 1.0992 - accuracy: 0.6536
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8474 - accuracy: 0.6771
Epoch 5/150
77/77 [==============================] - 0s 3ms/step - loss: 0.8449 - accuracy: 0.6745
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8763 - accuracy: 0.6810
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7835 - accuracy: 0.6745
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7977 - accuracy: 0.6198
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7260 - accuracy: 0.6693
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7042 - accuracy: 0.6680
Epoch 11/

* Comme on peut le voir dans le summary ci-dessous, notre modèle a bien des couches qui ne sont pas complètement connectées. 
* On voit que dense_2 et dense_3 ont 2 neurones en output, et concatenate récupère ces 4 neurones pour créer une nouvelle couche connecté ensuite à l'output

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 8)            72          ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 8)            72          ['dense[0][0]']                  
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 4)           0           ['dense_1[0][0]']                
 ingOpLambda)                                                                                 

In [22]:
model.layers[1].weights

#ici on voit les poids attribués à chaque neurone connecté à ceux de la 1ere couche cachée
#par exemple : [ 0.27119082,  0.33265215,  0.01492959, -0.02809624,  0.5031679 ,0.5641654 ,  0.19356844,  1.0565124 ]
#sont les poids de chaque neurone de la première couche lorsqu'ils arrivent au premier neurone de la seconde couche

[<tf.Variable 'dense/kernel:0' shape=(8, 8) dtype=float32, numpy=
 array([[ 0.27119082,  0.33265215,  0.01492959, -0.02809624,  0.5031679 ,
          0.5641654 ,  0.19356844,  1.0565124 ],
        [-0.5464286 , -0.33864352, -0.43449026, -0.48312673, -0.50841486,
          0.58769953,  0.30429852,  0.5709839 ],
        [-0.3959679 ,  0.04247158, -0.5819971 ,  0.35832328,  0.45101315,
         -0.36512387,  0.3865559 , -0.5063094 ],
        [ 0.2729497 , -0.19124335,  0.17686617, -0.23776406,  0.5030252 ,
          0.6347281 ,  0.15935485,  0.05759012],
        [-0.12576714, -0.02605964, -0.46710867,  0.23634996, -0.65293837,
          0.5676584 , -0.0899451 , -0.18737018],
        [-0.35165092,  0.38207075,  0.02477038, -0.06703442,  0.16760878,
          0.62183803,  0.43783376, -0.27949828],
        [ 0.17420238, -0.43965927, -0.3807923 , -0.5432582 ,  1.047496  ,
         -0.25603417, -0.15780833,  1.0869747 ],
        [ 0.5452575 ,  0.4397475 ,  0.42003745, -0.41930148, -0.03986422,

In [32]:
model.layers[5].weights
#pour la couche note fully connected, on remarque qu'on a pas une liste n x n comme plus haut avec 8 listes et 8 élément par listes
#on a bien 4 listes car on a les infos de 4 neurones
#mais dans chaque liste on a que 2 neurones qui sont connectés 
#par exemple : [ 0.18257232, -0.7017965 ]
# sont les poids des deux premiers neurones de la couche précédentes qui sont connectés au premier neurone de cette couche

[<tf.Variable 'dense_2/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.18257232, -0.7017965 ],
        [ 0.04232398,  0.4505998 ],
        [-0.5945198 ,  0.2949081 ],
        [-0.5803667 ,  0.3988894 ]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(2,) dtype=float32, numpy=array([-0.02651062,  0.01546192], dtype=float32)>]

In [ ]:
#-------------------- Initialisation du modèle -------------------------#
# couche d'input : 
input = keras.layers.Input(shape=(8,))

# couches cachées
hidden1 = Dense(8, activation='relu')(input)
hidden2 = Dense(8, activation='sigmoid')(hidden1)
hidden3_1 =  Dense(2)(hidden2[:,0:4])  # prend les 4 premiers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones
hidden3_2 =  Dense(2)(hidden2[:,4:])  # prend les 4 derniers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones
hidden_3 = keras.layers.concatenate([hidden3_1, hidden3_2])  
#la troisieme couche cachée possède maintenant ses 2 premiers neurones connectés au 4 premiers de la couche précédente
#et ces 2 derniers neurones connectés aux 4 derniers de la couche précédente

# output
output = Dense(1)(hidden_3) 

#on concatene les couches d'output pour en faire une seule couche output
model = keras.models.Model(input, output)
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)
#----------------------------------------------------------------------#


# Algorithme evolutionnaire

* **MAXGEN** = Nombre maximum de générations = nombre max de population de réseaux de neurones testées
* **POPSIZE** = Taille de la population = Nombre de réseau de neurones dans la population (d'individus)
* **PMUT** = Probabilité d'une mutation
* **PX** = Probabilité d'une recombination
* **N** = Taille d'un génôme = nombre de couches du réseaux

J'ai décidé de faire l'opération de mutation de la manière suivante :
 - l'ajout ou retrait de neurones se fera sur les deux premieres couches cachées
 - l'ajout de connection se fera sur la 3 eme couche cachée qui est not fully connected

In [52]:
round(5/2)

2

In [96]:
import random

#-------------------- CONSTANTES DE DEPART -------------------------#
MAXGEN = 20    # Nombre maximum de générations

POPSIZE = random.choice(range(20,100))   #taille de la population, pour avoir une valeur entre 20 et 100 individus

PMUT = 0.1     # probabilité d'une mutation ajout/retrait de neurones et connection

PX = 0.6        # probabilité d'une recombinaison

#N = la longueur d'un génome c'est à dire le nombre de couche de chaque réseaux


#---- Affichage des variables de départ ----#
print("Nombre max de génération :", MAXGEN)
print("Taille de la population :", POPSIZE)
print("Probabilité d'une mutation :", PMUT)
print("Probabilité d'une recombinaison :", PX)

#--------------------------  FITNESS   ------------------------------#
def fitness(model):# evaluation du model avec la loss 
    loss = model.evaluate(X,y)[0]
    return 1/(1+loss)

#------------------------- SELECTION ---------------------------------#
def selection(pop):
    f = [fitness(pop[i]) for i in range(len(pop))]
    cum = f.copy()
    for i in range(1,len(cum)):
        cum[i] += cum[i - 1]
    offspring = []
    for count in range(len(pop)): 
        r = random.uniform(0, cum[-1]) #on ne peut pas utiliser random.randrange sur des valeurs inférieurs à 1
        i = 0                          #notre calcul de fitness cumulé contient des valeurs qui le sont c'est pourquoi j'utilise random.uniform 
        while cum[i] < r:
            i += 1
        offspring.append(pop[i])
    return offspring

#-------------------------- MUTATION --------------------------------#
def mutation(pop):
    for i in range(len(pop)):
        if random.random()<PMUT:
            input = keras.layers.Input(shape=(8,))

            nbHidden1 = random.randrange(5,20)
            nbHidden2 = random.randrange(5,20)
            hidden1 = Dense(nbHidden1, activation='relu')(input)
            hidden2 = Dense(nbHidden2, activation='sigmoid')(hidden1)

            n= random.randrange(nbHidden2)
            hidden3_1 =  Dense(2)(hidden2[:,0:n]) 
            hidden3_2 =  Dense(2)(hidden2[:,n:])  
            hidden_3 = keras.layers.concatenate([hidden3_1, hidden3_2])  
    
            output = Dense(1)(hidden_3) 

            model = keras.models.Model(input, output)

            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

            model.fit(X, y, epochs=150, batch_size=10,verbose=0)
            
            pop[i] = model



#--------------------------- CROSSOVER ------------------------------#
#def crossover(mom, dad):
#    point = random.randrange(len(mom))
#    tmp = mom[point:]
#    mom[point:] = dad[point:]
#    dad[point:] = tmp

#------------------------- INITILISATION -----------------------------#
random.seed()

#--------- On cré notre population de réseaux de neurones --------#

# ils auront tous 8 neurones en entrée, un neurone en sortie
# et 3 couches cachées 
# il y a 5 couches au total 
pop = []
for i in range(POPSIZE):
    input = keras.layers.Input(shape=(8,))

    hidden1 = Dense(8, activation='relu')(input)
    hidden2 = Dense(8, activation='sigmoid')(hidden1)
    hidden3_1 =  Dense(2)(hidden2[:,0:4])  # prend les 4 premiers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones
    hidden3_2 =  Dense(2)(hidden2[:,4:])  # prend les 4 derniers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones
    hidden_3 = keras.layers.concatenate([hidden3_1, hidden3_2])  
    #la troisieme couche cachée possède maintenant ses 2 premiers neurones connectés au 4 premiers de la couche précédente
    #et ces 2 derniers neurones connectés aux 4 derniers de la couche précédente

    # output
    output = Dense(1)(hidden_3) 

    #on concatene les couches d'output pour en faire une seule couche output
    model = keras.models.Model(input, output)
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=150, batch_size=10,verbose=0)

    ind = model
    pop.append(ind)

#----------------------------------------------------------------------#

#---- Affichage des variables de départ ----#
print("Nombre max de génération :", MAXGEN)
print("Taille de la population :", POPSIZE)
print("Probabilité d'une mutation :", PMUT)
print("Probabilité d'une recombinaison :", PX)
#print("Longueur d'un génome :", N)

#---- Lancement de l'algorithme évolutionnaire ----#
print("Initial Population:")
bestGeneration = 0
bestInd = 0
bestFitness = 0

for g in range(MAXGEN):
    pop = selection(pop)
    mutation(pop)    
    #for i in range(0,POPSIZE,2):
    #    if random.random() < PX:
    #        crossover(pop[i],pop[i + 1])

    f = [fitness(pop[i]) for i in range(len(pop))]
    if max(f)>bestFitness :
        bestFitness = max(f)
        bestGenerationNumber = g
        bestGeneration = pop
        bestIndex = f.index(max(f))
        bestInd = pop[bestIndex]
        bestLoss = bestInd.evaluate(X,y)[0]

    print("--------------- Generation", g," --------------------- ")
    if max(f) == 1: #on arrete l'algorithme si on atteint notre fitness max, ici 1 
                    #il n'y a pas de else, car on continue jusqu'à MAXGEN, et on affichera la derniere 
                    #génération si on obtient pas une fitness de 1 avant
        print("------------ Solution found ------------")
        break


#-------------- Resultats ------------#

print("------------ Best Generation :", bestGenerationNumber," ------------")

print("Fitness atteinte: ", bestFitness) 
print("Loss correspondante: ", bestLoss) 
print("Genome de l'individu numero ",bestIndex," :") #numero de ligne de l'individu dans la population de la derniere génération
print(bestInd.summary())

Nombre max de génération : 20
Taille de la population : 71
Probabilité d'une mutation : 0.1
Probabilité d'une recombinaison : 0.6
Nombre max de génération : 20
Taille de la population : 71
Probabilité d'une mutation : 0.1
Probabilité d'une recombinaison : 0.6
Initial Population:
24/24 [==============================] - 0s 1ms/step - loss: 0.4977 - accuracy: 0.7344
--------------- Generation 0  --------------------- 
24/24 [==============================] - 0s 1ms/step - loss: 0.5590 - accuracy: 0.6992
--------------- Generation 1  --------------------- 
24/24 [==============================] - 0s 739us/step - loss: 0.6099 - accuracy: 0.6836
--------------- Generation 2  --------------------- 
24/24 [==============================] - 0s 2ms/step - loss: 0.5456 - accuracy: 0.7266
--------------- Generation 3  --------------------- 
24/24 [==============================] - 0s 2ms/step - loss: 0.5626 - accuracy: 0.7031
--------------- Generation 4  --------------------- 
24/24 [===========

In [ ]:
model.layers[0]

# test


In [53]:

pop = []
for i in range(3):
    input = keras.layers.Input(shape=(8,))

    hidden1 = Dense(8, activation='relu')(input)
    hidden2 = Dense(8, activation='sigmoid')(hidden1)
    hidden3_1 =  Dense(4)(hidden2[:,0:4])  # prend les 4 premiers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones
    hidden3_2 =  Dense(4)(hidden2[:,4:])  # prend les 4 derniers neurones de la couche caché 2
                                       # pour les mettre dans 2 neurones
    hidden_3 = keras.layers.concatenate([hidden3_1, hidden3_2])  
    #la troisieme couche cachée possède maintenant ses 2 premiers neurones connectés au 4 premiers de la couche précédente
    #et ces 2 derniers neurones connectés aux 4 derniers de la couche précédente

    # output
    output = Dense(1)(hidden_3) 

    #on concatene les couches d'output pour en faire une seule couche output
    model = keras.models.Model(input, output)
    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit the keras model on the dataset
    model.fit(X, y, epochs=150, batch_size=10)

    ind = model
    pop.append(ind)

Epoch 1/150
77/77 [==============================] - 1s 2ms/step - loss: 9.5964 - accuracy: 0.3490
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 9.5220 - accuracy: 0.3503
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 9.5003 - accuracy: 0.3490
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 9.4982 - accuracy: 0.3490
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 9.4829 - accuracy: 0.3490
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 9.4265 - accuracy: 0.3516
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 9.4243 - accuracy: 0.3516
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 5.2605 - accuracy: 0.4674
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8437 - accuracy: 0.6576
Epoch 10/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6821 - accuracy: 0.6562
Epoch 11/

In [37]:
pop[0].layers[1].weights

[<tf.Variable 'dense_5/kernel:0' shape=(8, 8) dtype=float32, numpy=
 array([[-0.15249328,  0.50063545,  0.28822315,  0.77834857, -0.20870301,
          0.70995283,  0.06436356,  0.5364591 ],
        [-0.19149394,  0.4194183 ,  0.43271613,  0.00673244,  0.20345113,
         -0.50599587, -0.17756386, -0.64592683],
        [-0.15060905, -0.01711289, -0.16999029, -0.54341507,  0.3362368 ,
         -0.6319333 , -0.20955268,  0.14819115],
        [ 0.22643656,  0.38280573, -0.32989797, -0.21800256,  0.02763058,
          0.17404403,  0.3200085 , -0.29068407],
        [ 0.57609224,  0.5193932 ,  0.35470444, -0.5800949 ,  0.5095289 ,
          0.5447184 , -0.56283355,  0.23922133],
        [-0.27729926,  0.20399585,  0.33614758,  0.25263458, -0.31825197,
          0.31686515, -0.0670006 , -0.48025194],
        [-0.20730674, -0.22139741,  0.85036033,  0.59092706, -0.25549826,
          0.06190499,  0.3557751 ,  0.04220556],
        [-0.07651555, -0.26040524,  0.4634209 ,  0.6040928 , -0.053823 

In [69]:
def mutation(pop):
    for i in range(len(pop)):
            input = keras.layers.Input(shape=(8,))

            nbHidden1 = random.randrange(5,20)
            nbHidden2 = random.randrange(5,20)
            hidden1 = Dense(nbHidden1, activation='relu')(input)
            hidden2 = Dense(nbHidden2, activation='sigmoid')(hidden1)

            n= round(nbHidden2/2)
            hidden3_1 =  Dense(2)(hidden2[:,0:n]) 
            hidden3_2 =  Dense(2)(hidden2[:,n:])  
            hidden_3 = keras.layers.concatenate([hidden3_1, hidden3_2])  
    
            output = Dense(1)(hidden_3) 

            model = keras.models.Model(input, output)

            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

            model.fit(X, y, epochs=150, batch_size=10)
            
            pop[i] = model

print(pop[2].summary())
mutation(pop)
print(pop[2].summary())



Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 8)]          0           []                               
                                                                                                  
 dense_55 (Dense)               (None, 8)            72          ['input_12[0][0]']               
                                                                                                  
 dense_56 (Dense)               (None, 8)            72          ['dense_55[0][0]']               
                                                                                                  
 tf.__operators__.getitem_22 (S  (None, 4)           0           ['dense_56[0][0]']               
 licingOpLambda)                                                                           


Nous créons un modèle séquentiel et ajoutons des couches une par une jusqu'à ce que nous soyons satisfaits de notre architecture réseau. 
Nous commencerons avec une structure de réseau entièrement connectée avec trois couches : 


    Le modèle attend des lignes de données avec 8 variables (l' input_dim=8 argument )
    La première couche cachée a 12 nœuds et utilise la fonction d'activation relu.
    La deuxième couche cachée a 8 nœuds et utilise la fonction d'activation relu.
    La couche de sortie a un nœud et utilise la fonction d'activation sigmoïde.


Les couches entièrement connectées sont définies à l'aide de la classe Dense . Nous pouvons spécifier le nombre de neurones ou de noeuds dans la couche comme premier argument et spécifier la fonction d'activation à l'aide de l'argument du même nom : 


In [33]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

Dans ce cas, nous utiliserons l'entropie croisée comme loss argument. Ce calcul de loss est fait pour les problèmes de classification binaire et est définie dans Keras comme « binary_crossentropy ».

In [34]:

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

La formation se déroule sur des époques et chaque époque est divisée en lots.

    Epoch  : un passage à travers toutes les lignes de l'ensemble de données d'apprentissage.
    Batch : Un ou plusieurs échantillons pris en compte par le modèle dans une époque avant que les poids ne soient mis à jour.


In [35]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 4s 2ms/step - loss: 4.3558 - accuracy: 0.5833
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 0.9588 - accuracy: 0.6068
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 0.8454 - accuracy: 0.6341
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7620 - accuracy: 0.6406
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7504 - accuracy: 0.6732
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7179 - accuracy: 0.6732: 0s - loss: 0.7016 - accuracy: 0.69
Epoch 7/150
77/77 [==============================] - ETA: 0s - loss: 0.8701 - accuracy: 0.62 - 0s 1ms/step - loss: 0.8010 - accuracy: 0.6250
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7101 - accuracy: 0.6732
Epoch 9/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7088 - accuracy: 0.6901
Epoch 10/150
77/77 [===========

In [37]:
# evaluate the keras model
loss, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))
print('Loss : %.2f' % loss)

24/24 [==============================] - 0s 1ms/step - loss: 0.5099 - accuracy: 0.7643
Accuracy: 76.43
Loss : 0.51


In [54]:
# AUTRES TEST : NOT FULLY CONNECTED
#  On définie le modele 

# couche d'input : 
input = keras.layers.Input(shape=(8,))

# couches cachées
hidden1 = Dense(8, activation='relu')(input)
hidden2 = Dense(8, activation='sigmoid')(hidden1)
hidden3_1 =  Dense(4)(hidden2[:,0:4])  # prend les 3 premiers neurones de la couche caché 2
hidden3_2 =  Dense(4)(hidden2[:,4:])  # prend les 3 derniers neurones de la couche caché 2
hidden_3 = keras.layers.concatenate([hidden3_1, hidden3_2])  

# output
output = Dense(1)(hidden_3) 

#on concatene les couches d'output pour en faire une seule couche output
model = keras.models.Model(input, output)

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, y, epochs=200, batch_size=10)

# evaluate the keras model
loss, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))
print('Loss : %.2f' % loss)

Epoch 1/200
77/77 [==============================] - 1s 2ms/step - loss: 0.6748 - accuracy: 0.6419
Epoch 2/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6543 - accuracy: 0.6497
Epoch 3/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6496 - accuracy: 0.6510
Epoch 4/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6502 - accuracy: 0.6536
Epoch 5/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6655 - accuracy: 0.6497
Epoch 6/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6457 - accuracy: 0.6549
Epoch 7/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6455 - accuracy: 0.6562
Epoch 8/200
77/77 [==============================] - 0s 2ms/step - loss: 0.7106 - accuracy: 0.6536
Epoch 9/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6688 - accuracy: 0.6419
Epoch 10/200
77/77 [==============================] - 0s 2ms/step - loss: 0.6442 - accuracy: 0.6589
Epoch 11/

# Memo 

* Batch size = PopSize (taille de la population testée)
* Epoch = MaxGen (Nombre max de génération pour trouver la solution)

FULLY CONNECTED LAYERS MODEL : 

In [42]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

# evaluate the keras model
loss, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))
print('Loss : %.2f' % loss)

Epoch 1/150
77/77 [==============================] - 1s 1ms/step - loss: 7.3705 - accuracy: 0.5391
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 1.1026 - accuracy: 0.6068
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8003 - accuracy: 0.6549
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7186 - accuracy: 0.6654
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6721 - accuracy: 0.6771
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6599 - accuracy: 0.6719
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6721 - accuracy: 0.6732
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6619 - accuracy: 0.6784
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6415 - accuracy: 0.6992
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6424 - accuracy: 0.6966
Epoch 11/